In [47]:
from EMS import EMS
from Organization import Organization
from Energy import Energy
from Hierarchy import Hierarchy
from Dataflow import DataFlow
import pandas as pd
import numpy as np

from EMS import EMS
from Organization import Organization
from Energy import Energy
from Hierarchy import Hierarchy
from Dataflow import DataFlow
import xml.etree.ElementTree as ET
from common import readOption
from IPython.display import display
import os
from pathlib import Path
# Set the maximum number of columns to be displayed to None
pd.set_option('display.max_columns', None)

In [48]:
# Create sample DataFrames
sys_df = pd.DataFrame({'id': [1, 2, 3], 'name': ['Alice', 'Bob', 'Charlie'], 'col1': ['a', 'b', 'c']})
dp = pd.DataFrame({'ref_id': [2, 4, 5], 'name': ['Bob', 'David', 'Emily'], 'col2': ['x', 'y', 'z']})

# Merge DataFrames using multiple columns for matching
merged_df = pd.merge(left=sys_df, 
                    right=dp, 
                    how='left',
                    left_on=['id', 'name'], 
                    right_on=['ref_id', 'name'])



print(merged_df)


   id     name col1  ref_id col2
0   1    Alice    a     NaN  NaN
1   2      Bob    b     2.0    x
2   3  Charlie    c     NaN  NaN


In [49]:
filepath = "./output/isf/meters.csv"
meters = pd.read_csv(filepath, index_col=False)

filepath = "./output/isf/sys_df.csv"
sys_df = pd.read_csv(filepath, index_col=False)

filepath = "./output/isf/meter_dp_df.csv"
meter_dp_df = pd.read_csv(filepath, index_col=False)

sys_dp_df = pd.merge(left=sys_df, 
                    right=meter_dp_df, 
                    how='left',
                    left_on=['meter_id', 'source_key'], 
                    right_on=['meter_ref_id', 'source_key'])
sys_dp_df[:10]

,id,parent_system_id,system_name,source_key,meter_id,composition_expression,component_of_id,company_id,city_id,city_name,meter_ref_id,dp_id,ref_id
0,15613,11398.0,Air Conditioning (Indoor Units),NaN,NaN,0 if ({id_17938}) <0.002 else ({id_17917}+{id...,NaN,307,2.0,Hong Kong,NaN,NaN,NaN
1,15570,NaN,isf_10RAC1_4112,isf_10RAC1_4112,536.0,NaN,15580.0,307,2.0,Hong Kong,536.0,94977056-6f71-4d2a-8cf4-f57c6d664ec1,15570.0
2,9805,NaN,isf_5A4B_4148,isf_5A4B_4148,536.0,NaN,7555.0,307,2.0,Hong Kong,536.0,075d4875-24b7-4862-8af2-e26695aeb34d,9805.0
3,16309,16307.0,2/F-5/F Classrooms,NaN,NaN,NaN,NaN,307,2.0,Hong Kong,NaN,NaN,NaN
4,18791,7666.0,309 Lights Total,309 Lights Total,1187.0,NaN,NaN,307,2.0,Hong Kong,1187.0,910ea2eb-4118-432b-9c5e-db636be1a10c,18791.0
5,18790,7661.0,304 Lights Total,304 Lights Total,1187.0,NaN,NaN,307,2.0,Hong Kong,1187.0,ccd28692-bf23-47a5-9218-7fcca0ded653,18790.0
6,15592,11329.0,Air Conditioning (Indoor Units),NaN,NaN,{id_15521}+{id_15522}+{id_15523}+{id_15524}+{i...,NaN,307,2.0,Hong Kong,NaN,NaN,NaN
7,9807,NaN,isf_5A4B_4152,isf_5A4B_4152,536.0,NaN,7555.0,307,2.0,Hong Kong,536.0,d6856d4d-f2ab-4468-b2b6-c0c9fbab68a2,9807.0
8,9743,NaN,isf_6A4B_4116,isf_6A4B_4116,536.0,NaN,7584.0,307,2.0,Hong Kong,536.0,2fe1527f-9f71-4a61-8a75-74ab25e97623,9743.0
9,9744,NaN,isf_6A4B_4152,isf_6A4B_4152,536.0,NaN,7584.0,307,2.0,Hong Kong,536.0,40a1ef7e-2a3f-4b30-8a5f-da1bf8baf481,9744.0


In [50]:
sys_df[:3]

,id,parent_system_id,system_name,source_key,meter_id,composition_expression,component_of_id,company_id,city_id,city_name
0,15613,11398.0,Air Conditioning (Indoor Units),NaN,NaN,0 if ({id_17938}) <0.002 else ({id_17917}+{id...,NaN,307,2.0,Hong Kong
1,15570,NaN,isf_10RAC1_4112,isf_10RAC1_4112,536.0,NaN,15580.0,307,2.0,Hong Kong
2,9805,NaN,isf_5A4B_4148,isf_5A4B_4148,536.0,NaN,7555.0,307,2.0,Hong Kong


In [51]:
meter_dp_df[:3]

,meter_ref_id,source_key,dp_id,ref_id
0,229,12f lp,f8349f39-7487-43a5-af47-ba9b655d221b,1072.0
1,229,12f ac,2b0aa5d8-4135-464f-b3d7-a1f138c2b863,1243.0
2,202,Window Side Sockets & Printer,b2148ed7-923d-420d-a582-fccb05180f5a,1742.0


In [52]:
# # Check for rows with empty 'source_key' after dropna
# empty_source_key_mask = meter_dp_df['source_key'].isna()

# # Get the index of the row with empty 'source_key'
# empty_source_key_row_index = empty_source_key_mask.idxmax()

# # Print the index or use it to access the row
# print(f"Row with empty 'source_key': {empty_source_key_row_index}")


In [ ]:
meter_dp_df.shape

In [ ]:
# # Create a mask selecting rows where 'source_key' is not 'nan'
# # not_nan_source_key_mask = meter_dp_df['source_key'] == 'nan'
# not_nan_source_key_mask = ~meter_dp_df['source_key'].isna()

# # Filter the dataframe using the mask
# meter_dp_df = meter_dp_df[not_nan_source_key_mask]
# meter_dp_df.shape


In [ ]:
print(meter_dp_df.isnull().sum())
print(meter_dp_df.shape)

In [ ]:
site_path = "/home/zhixiang/EnTrak/Project/ems-hierarchy-energy/output/isf"
meter_dp_df.to_csv(f"{site_path}/meter_dp_df2.csv", index=False)

In [ ]:
# Drop rows with any NaN values
meter_dp_df.dropna(inplace=True)
meter_dp_df.isnull().sum() 

In [ ]:
import binascii

byte_string = b"\210!\327H\215\030A{\251\027\224\036M\371\323\311"  # Convert from string to bytes
uuid_bytes = byte_string #binascii.unhexlify(byte_string)  # Decode byte string into bytes
import uuid

uuid_object = uuid.UUID(bytes=uuid_bytes)
uuid_string = str(uuid_object)
print(uuid_string)


byte_string = b'\xe2\x8b`\x98C\xa1M\xee\x8bP,\xa8\xf8\xdfy\xb6'  # Convert from string to bytes
uuid_bytes = byte_string #binascii.unhexlify(byte_string)  # Decode byte string into bytes


uuid_object = uuid.UUID(bytes=uuid_bytes)
uuid_string = str(uuid_object)
print(uuid_string)


In [ ]:
codes = readOption("codes")
codes_list = codes.split(",")
def create_site_logger(codes_list = None):
    for code in codes_list:
        site_path = f"./output/{code}"
        if not Path(f"{site_path}").is_dir(): 
            os.makedirs(site_path)

In [ ]:
# Specify the path to your CSV file
# filepath = "./output/vtchk/sys_df_after.csv"
# filepath = "./output/vtchk/sys_df.csv"
filepath = "./output/hkcec/result_df.csv"



# Read the CSV file into a dataframe using read_csv()
df = pd.read_csv(filepath)
df[:3]

In [ ]:
def remove_extra_spaces(text):
  """Removes extra spaces from a string, including leading, trailing, and multiple spaces between words.

  Args:
      text: The string to process.

  Returns:
      A new string with extra spaces removed.
  """
  return " ".join(text.split()).strip()

def is_none_or_nan(value):
  return value is None or pd.isnull(value) or value == 'nan' 

def is_none_or_nan_zero(value):
  return value is None or pd.isnull(value) or len(value) == 0 or remove_extra_spaces(str(value)) == '' or value == 'nan'

In [ ]:
sys_df = df
# site
for i in range(len(sys_df)):
    if np.isnan(sys_df.loc[i, 'parent_system_id']) \
        and is_none_or_nan(sys_df.loc[i, 'component_of_id']):         
            print(sys_df.loc[i, "name_x"])

In [ ]:
# pov
# folder system which has no meterid, defined as hierachy.node_pov
for i in range(len(sys_df)):
    if not is_none_or_nan(sys_df.loc[i, 'parent_system_id']) \
        and is_none_or_nan_zero(str(sys_df.loc[i, 'composition_expression'])) \
        and is_none_or_nan_zero(str(sys_df.loc[i, 'meter_id'])):
            print(f"{sys_df.loc[i, 'id_x']} {sys_df.loc[i, 'name_x']}")

In [ ]:
for i in range(len(sys_df)):
    if not is_none_or_nan(sys_df.loc[i, 'parent_system_id']):
        ot = f"[POV] id [{sys_df.loc[i, 'id_x']}]"
        ce = str(sys_df.loc[i, 'composition_expression'])
        print(f"{ot} ce:{ce}, check-ce:{is_none_or_nan_zero(ce)}")
        mi = str(sys_df.loc[i, 'meter_id'])
        print(f"{ot} mi:{mi}, check-mi:{is_none_or_nan_zero(mi)}")

In [ ]:
# data system which has data(kwh), defined as hierachy.node_datapoint(data_type==energy) and energy.energy_datapoint                
for i in range(len(sys_df)):
    if not is_none_or_nan(sys_df.loc[i, 'meter_id']) \
        and not is_none_or_nan_zero(sys_df.loc[i, 'source_key']):
        # and len(sys_df.loc[i, 'source_key']) > 0):       

        meter_id = sys_df.loc[i, 'meter_id']
        source_key = sys_df.loc[i, 'source_key']
        sys_name = sys_df.loc[i, 'name_x']

        _df = sys_df[(sys_df['meter_id']==meter_id) & \
                            (sys_df['source_key']==source_key) & \
                            (sys_df['ref_id']!=np.nan) ]
        
        _ref_id, _energy_datapoint_id = None, None
        if _df.shape[0] == 0:
            # 说明此sourcekey对应的所有systems都没有一个在energy.datapoint中出现
            # 创建一个新的energy.datapoint
            _ref_id = sys_df.loc[i, 'id_x']
            name = sys_df.loc[i, "name_x"]
            meter_id = sys_df.loc[i, "meter_id"]

            print(f"not found abnormal system:[{_ref_id}] name:[{name}] in energy.datapoint")

        if _df.shape[0] >= 1:
            _df2 = _df[(~pd.isnull(_df['id_x'])) & (~pd.isnull(_df['id_y']))]            

            if _df2.shape[0] == 0:                
                print(f"ID [{sys_df.loc[i, 'id_x']}] meter:[{int(meter_id)}] source_key:[{source_key}] name: [{sys_name}] dosnt find energy_datapoint.id")
                _energy_datapoint_id = None
                _ref_id = None
                # 当前这个system在energy_datapoint里边没找到记录，
                # 则取相同meterid和sourcekey的system对应的第一个energy_datapoint的记录                   
                # _df3 = sys_df[(sys_df['source_key']==source_key)] 
                _df3 = sys_df[(sys_df['meter_id']==meter_id)] 
                print(_df3[['source_key', 'ref_id', 'id_y']])

                for _, row in _df3.iterrows():
                    # print(row)
                    if not is_none_or_nan(row['id_y']):
                        _energy_datapoint_id = row['id_y']
                        _ref_id = row['ref_id']
                        print(f"ID [{sys_df.loc[i, 'id_x']}] will use sysID:[{_ref_id}] with energy_datapoint.id:{_energy_datapoint_id}")
                        break
                
                if _energy_datapoint_id is None:
                    print(f"ID [{sys_df.loc[i, 'id_x']}] didn't find any energy.datapoint_id for replace it.")
                
                del _df3

In [ ]:
code = readOption("code")
components_binding = readOption("components_binding")        
host=readOption("databases.organization.host")
port=readOption("databases.organization.port")
database=readOption("databases.organization.database")
user=readOption("databases.organization.username")
password=readOption("databases.organization.password")
org = Organization(host=host, port=port, user=user, password=password, database=database)

tenant = org.Tenant(code)

print(f"{tenant.name.values[0]} | {tenant.company_code.values[0]} | {tenant.id.values[0]}")

In [ ]:
# generate tenant tree in dataflow
dataFlow = DataFlow(code, components_binding, simulation=True)
df = dataFlow.PreparingData()
# df = dataFlow.LoadData()
logger.info("====================== create_nodes_and_datapoints ==========================")
dataFlow.create_nodes_and_datapoints(df, tenant.id.values[0])

In [ ]:
ssd = dataFlow.hr.simulation_sys_df
print(ssd.shape)
with pd.option_context('display.max_columns', None):
    display(ssd[:2])

In [ ]:
srd = dataFlow.hr.simulation_relations_df
print(srd.shape)
with pd.option_context('display.max_columns', None):
    display(srd[:10])

In [ ]:
site_node_id = srd[srd['parent_type']=='SITE']['parent_id'].iloc[0]
print(site_node_id)

In [ ]:
hr = dataFlow.hr
logger.debug("====================== Simulate TenantTree XML ==========================")    
tenantTree = dataFlow.hr.TenantSimulateTree(
                        # tenant.id.values[0],
                        site_node_id,
                        tenant.name.values[0],
                        tenant.company_code.values[0])

site_path = f"./output/{code}"
hr.SaveToXml(tenantTree, f"{site_path}/sim_new_{code}.xml") 

In [ ]:
host=readOption("databases.ems.host")
port=readOption("databases.ems.port")
database=readOption("databases.ems.database")
user=readOption("databases.ems.username")
password=readOption("databases.ems.password")    

logger.debug("======================= EMS system tree XML =========================")
ems = EMS(host=host, port=port, user=user, password=password, database=database)
company = ems.company(code)

# logger.debug(f"{company.name.values[0]} | {company.code.values[0]} | {company.id.values[0]}")
companyTree = ems.SystemTree(company.id.values[0],
                                company.name.values[0],
                                company.code.values[0])

ems.SaveToXml(companyTree, f"{site_path}/sim_{code}_system.xml")

In [ ]:
ems = EMS("localhost", "5432", "mangopie", "mangopie", "mangopie")

In [ ]:
company_df = ems.company(code="cdnis")
print(company_df)

In [ ]:
org = Organization()
tenant_df = org.Tenant(code="cdnis")

print(tenant_df)

In [ ]:
joined_tenant = pd.merge(company_df, tenant_df, how="left", left_on="code", right_on="company_code")
tenant_df = joined_tenant[['id_x', 'id_y', 'name_x', 'code']]

new_names = {'id_x': 'company_id', 'id_y': 'tenant_id', 'name_x': 'company', "code": 'company_code'}
tenant_df = tenant_df.rename(columns=new_names)

with pd.option_context('display.max_columns', None):
    display(tenant_df[:3])   

In [ ]:
sys_df = ems.systems(code="cdnis")
# Set options temporarily within the context of `display()`
with pd.option_context('display.max_columns', None):
    display(sys_df[:8])

In [ ]:
# sys_df = ems.virtual_systems(code="cdnis")
# # Set options temporarily within the context of `display()`
# with pd.option_context('display.max_columns', None):
#     display(sys_df[:8])

In [ ]:
energy = Energy("localhost", "5432", "energy", "energy", "energy")

In [ ]:
dp = energy.dataPoint(columns = ["id", "ref_id", "name"])
dp[:3]

In [ ]:
joined_df = pd.merge(sys_df, dp, how="left", left_on="id", right_on="ref_id")


# 查看连接后的DataFrame
with pd.option_context('display.max_columns', None):
    display(joined_df[:8])


In [ ]:
joined_df.loc[joined_df['id_x'] == int(386)]#[['id_y', 'ref_id']]

In [ ]:
joined_df.loc[joined_df['source_key'] == 'ML']

In [ ]:
with pd.option_context('display.max_columns', None):
    display(tenant_df[:3])   

In [ ]:
joined_df = pd.merge(joined_df, tenant_df, how="left", left_on="company_id", right_on="company_id")
# 查看连接后的DataFrame
with pd.option_context('display.max_columns', None):
    display(joined_df[:8])

In [ ]:
def replace_id(df):
  """
  循环替换 DataFrame 中的 {id_xxx}

  Args:
    df: 输入 DataFrame

  Returns:
    替换后的 DataFrame
  """
  import re
  for i in range(df.shape[0]):
    row = df.iloc[i]
    composition_expression = row['composition_expression']
    for match in re.finditer(r'{id_(\d+)}', composition_expression):
      id_xxx = match.group(1)
      use_system_id = df.loc[df['id'] == int(id_xxx), 'use_system_id'].values[0]
      composition_expression = composition_expression.replace(match.group(0), "{id_"+str(use_system_id)+"}")
    df.loc[i, 'composition_expression'] = composition_expression

  return df

# 使用示例
df = pd.DataFrame({
  'id': [13, 63, 8],
  'composition_expression': [
    '1 if  {id_13}  >= 3 else 1 if  {id_63}  >= 3 else {id_8}',
    '1 if ({id_63}*2)+({id_63}*{id_8})  >= 3 else {id_8}',
    '{id_8}'
  ],
  'use_system_id': [100, 200, 300]
})

print(df)
print("----------------------------------------------------------------")
df = replace_id(df)

print(df)


In [ ]:
vdp = energy.virtualDataPoint()
vdp[:3]

In [ ]:
sys_df

In [ ]:
hr = Hierarchy()

In [ ]:
n_dp = hr.nodeDataPoint()

with pd.option_context('display.max_columns', None):
    display(n_dp[:2])

In [ ]:
n_pov = hr.nodePov()
with pd.option_context('display.max_columns', None):
    display(n_pov[:2])

In [ ]:
n_site = hr.nodeSite()
with pd.option_context('display.max_columns', None):
    display(n_site[:2])

In [ ]:
n_relations = hr.nodeRelations()
with pd.option_context('display.max_columns', None):
    display(n_relations[:2])

In [ ]:
# tenantTree = hr.TenantTree("c93789d6-d78e-5382-8817-c3917ca879be")
# tenantTree